# Data Ingest of 10-h Fuel Moisture Content

This notebook demonstrates retrieval and filtering of 10-h dead FMC data from a RAWS stash maintained by Angel Farguell.

## References

For more info on python library API, see Brian Blaylock's `SynopticPy` [python package](https://github.com/blaylockbk/SynopticPy)

For more info on available Synoptic RAWS variables, see [Synoptic Data](https://demos.synopticdata.com/variables/index.html) documentation

## Setup

In [ ]:
import os.path as osp
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import synoptic
import json
import sys
import numpy as np
import polars as pl
import pandas as pd
sys.path.append('../src')
from utils import Dict, read_yml, read_pkl, str2time, print_dict_summary
from data_funcs import rename_dict
import ingest.retrieve_raws_api as rr
import ingest.retrieve_raws_stash as rrs

In [ ]:
# raws_meta = read_yml("../etc/variable_metadata/raws_metadata.yaml")

with open("../etc/training_data_config.json", "r") as json_file:
    config = json.load(json_file)   
    config = Dict(config)

In [ ]:
config

In [ ]:
raws_stash_path = rrs.raws_meta["raws_stash_path"]
print(raws_stash_path)
osp.exists(raws_stash_path)

In [ ]:
raws_dict = rrs.build_raws_dict(config)

In [ ]:
print_dict_summary(raws_dict)

In [ ]:
def lonlat_to_xy(pts):
    """
    Return a polars dataframe of RAWS sensor data and associated units.
    
    Parameters:
    -----------
    pts : pandas DataFrame 

    Returns:
    --------
    test : str
        test
    """ 

    
    return ds_pts

In [ ]:
from herbie import HerbieLatest

In [ ]:
ds = HerbieLatest(model="hrrr", product="prs", fxx=0).terrain()

In [ ]:
bbox = config.bbox
start = str2time(config.start_time)
end = str2time(config.end_time)
bbox_reordered = [bbox[1], bbox[0], bbox[3], bbox[2]]
sts = rr.get_stations(bbox_reordered, start, end)
points = pd.DataFrame(sts[["longitude", "latitude"]], columns=["longitude", "latitude"])
points

In [ ]:
points.shape

In [ ]:
ds_pts = ds.herbie.pick_points(points, method="nearest")
ds_pts

In [ ]:
from herbie.toolbox import EasyMap, pc
from herbie import paint

In [ ]:
ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.orog,
    transform=pc,
    cmap = paint.LandGreen.cmap,
)

In [ ]:
# ds_pts = ds_pts.expand_dims({"stid": sts["stid"].to_numpy()})
ds_pts = ds_pts.assign_coords(point = sts["stid"].to_numpy())

In [ ]:
ds_pts

In [ ]:
ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.orog,
    transform=pc,
    cmap = paint.LandGreen.cmap,
)

for i in ds_pts.point:
    z = ds_pts.sel(point=i)
    ax.scatter(z.longitude, z.latitude, color="k", transform=pc)
    ax.scatter(
        z.point_longitude, z.point_latitude, color="blue", marker=".", transform=pc
    )

In [ ]:
ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax

p = ax.pcolormesh(
    ds.longitude,
    ds.latitude,
    ds.orog,
    transform=pc,
    cmap = paint.LandGreen.cmap,
)

for i in ds_pts.point:
    z = ds_pts.sel(point=i)
    ax.scatter(z.longitude, z.latitude, color="k", transform=pc)
    ax.scatter(
        z.point_longitude, z.point_latitude, color="blue", marker=".", transform=pc
    )
    ax.text(
        z.point_longitude,
        z.point_latitude,
        f" {z.point.item()}",
        transform=pc,
    )

buff = 1
ax.set_extent([bbox[1]-buff, bbox[3]+buff, bbox[0]-buff, bbox[2]+buff])

In [ ]:
ds.x